In [3]:
import pandas
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats


In [12]:
def SSTbehav(infilename):
    
    # Import from a csv file with a header
    df=pandas.read_csv(infilename,skipfooter=1)
    
    # Re-name columns
    df.columns = ['Block','TrialType','Arrow','Response','leftSSD','rightSSD','ACC','RT']
    
    # Check the number of stop trials
    if np.sum(df.TrialType) != 100:
        print("Stop trial number is wrong! (" + str(np.sum(df.TrialType)) + ")")
    
    # Compare correct GO RT vs. Failed Stop RT
    # GO RT should be slower than Failed Stop RT
    GoTrials = df.loc[(df.TrialType==0) & (df.ACC==1), :]
    FsTrials = df.loc[(df.loc[:,'ACC']==4) & (df.loc[:,'RT']<1000), :] # failed stop RT<1000ms included
    GoRT = np.mean(GoTrials.RT)
    FsRT = np.mean(FsTrials.RT)
    
    # Check if p(STOP) converges to .5
    StopTrialNum = np.sum(df.TrialType) # get # of stop trials
    SuccessStopNum = np.sum(df.ACC == 3) # get # of failed stop trials
    p_stop = SuccessStopNum/StopTrialNum
    
    # SSRT (Stop Signal Reaction Time) calculation with replacement with Go omission trials
    # Make a new RT2 column
    df.loc[:,'RT2'] = df.RT

    # Replace Go omission (miss) trials with longest GO RT
    df.loc[df.ACC==0,'RT2'] = np.max(df.RT2)
    
    # Get p(response|STOPSIGNAL): probability of failed stop
    FailedStopNum = np.sum(df.ACC == 4) # get # of failed stop trials
    p_resp = FailedStopNum / StopTrialNum # calculate p(respond|STOP)
    
    # Get RT from GO distribution
    # Find n-th fastest RT, here n = p_resp * 100
    AllGoTrials = df.loc[(df.TrialType==0), :]
    NthRT = np.percentile(np.sort(AllGoTrials.RT), p_resp*100)
    
    # Get one SSD from each trial (if left arrow -> get leftSSD)
    for i in range(0,df.shape[0]):
        if df.Arrow[0] == 1:
            df.loc[i,'SSD'] = df.leftSSD[i]
        else:
            df.loc[i,'SSD'] = df.rightSSD[i]

    # Get all stop trials
    StopTrials = df.loc[df.loc[:,'TrialType']==1,:]

    # Get mean SSD (SSDs in Go trials don't mean anything )
    avgSSD = np.mean(StopTrials.SSD)
    
    # Calculate SSRT 
    # Also calculate SSRT using mean method (meanGO RT - mean SSD)
    SSRTint = NthRT - avgSSD #integration method : NthGO RT - mean SSD
    SSRTmean = GoRT - avgSSD #mean method: mean correct GO RT - mean SSD


    return GoRT, FsRT, p_stop, avgSSD, SSRTint, SSRTmean
    


In [13]:
infiles = glob.glob("./Subject*.csv")
infiles = np.sort(infiles)

data = {'SubID':[],'GoRT':[], 'FsRT':[], 'p_stop':[], 'avgSSD':[], 'SSRTint':[], 'SSRTmean':[]}
df = pandas.DataFrame(data=data)

for i in range(0,len(infiles)):
    GoRT, FsRT, p_stop, avgSSD, SSRTint, SSRTmean = SSTbehav(infiles[i])
    data = {'SubID':infiles[i][9:12],'GoRT':GoRT, 'FsRT':FsRT, 'p_stop':p_stop, 'avgSSD':avgSSD, 'SSRTint':SSRTint, 'SSRTmean':SSRTmean}
    df1 = pandas.DataFrame(data=data, index=[0])
    df = df.append(df1)
#print(df)

t, p = stats.ttest_rel(df.GoRT ,df.FsRT)
print("GO RT vs. Failed Stop RT")
print("t(" + str(df.shape[0]-1) + ") = " + f"{t:.3f}" + ", p = " + f"{p:.3f}")

print(df)

df.to_csv('GroupData.csv')

/opt/anaconda3/envs/psychopy/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


GO RT vs. Failed Stop RT
t(4) = 6.730, p = 0.003
  SubID        GoRT        FsRT  p_stop  avgSSD  SSRTint    SSRTmean
0   101  478.702564  465.829787    0.51   317.0   153.00  161.702564
0   102  419.171717  389.000000    0.50   211.5   190.00  207.671717
0   103  418.725888  390.411765    0.49   156.5   260.99  262.225888
0   104  448.715000  421.680000    0.50   231.0   220.50  217.715000
0   105  469.707071  431.913043    0.53   257.5   212.03  212.207071


In [8]:
infiles = np.sort(infiles)
print(infiles)

['./Subject101_10492004.csv' './Subject102_11522004.csv'
 './Subject103_12302004.csv' './Subject104_23182004.csv'
 './Subject105_22572604.csv']


In [9]:
df=pandas.read_csv(infiles[0],skipfooter=1)
    
# Re-name columns
df.columns = ['Block','TrialType','Arrow','Response','leftSSD','rightSSD','ACC','RT']
AllGoTrials = df.loc[(df.TrialType==0), :]
AllGoTrials.shape[0]

/opt/anaconda3/envs/psychopy/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


200